## Position using DeepLabCut from Scratch

In [ ]:
from pathlib import Path, PosixPath, PurePath
import os
import glob
import numpy as np
import pandas as pd
import pynwb
import datajoint as dj
import spyglass.common as sgc
import spyglass.position as sgp

### Table of Contents<a id='ToC'></a>
[`DLCProject`](#DLCProject)<br>
[`DLCModelTraining`](#DLCModelTraining)<br>
[`DLCModel`](#DLCModel)<br>
[`DLCPoseEstimation`](#DLCPoseEstimation)<br>
[`DLCSmoothInterp`](#DLCSmoothInterp)<br>
[`DLCCentroid`](#DLCCentroid)<br>
[`DLCOrientation`](#DLCOrientation)<br>
[`DLCPos`](#DLCPos)<br>
[`DLCPosVideo`](#DLCPosVideo)<br>
[`PosSource`](#PosSource)<br>
[`IntervalPositionInfo`](#IntervalPositionInfo)<br>

#### [DLCProject](#ToC) <a id='DLCProject'></a>

In [ ]:
sgp.DLCProject()

In [ ]:
sgp.BodyPart.insert([{'bodypart': 'greenLED', 'bodypart_description': 'green LED on implant LED ring'},
                     {'bodypart': 'redLED_C', 'bodypart_description': 'center red LED on implant LED ring'},
                     {'bodypart': 'redLED_L', 'bodypart_description': 'left red LED on implant LED ring'},
                     {'bodypart': 'redLED_R', 'bodypart_description': 'right red LED on implant LED ring'},
                     {'bodypart': 'tailBase', 'bodypart_description': 'base of the tail on subject'},],
                    skip_duplicates=True)

In [ ]:
video_list = [{'nwb_file_name': 'J1620210529_.nwb', "epoch": 2},
              {'nwb_file_name': 'peanut20201103_.nwb', "epoch": 4},]
team_name = "JG_DG"

In [ ]:
project_key = sgp.DLCProject.insert_new_project(
    project_name='scratch_test',
    bodyparts=['redLED_C', 'greenLED', 'redLED_L', 'redLED_R', 'tailBase'],
    lab_team=team_name,
    frames_per_video=100,
    video_list=video_list,
)
project_key['project_path'] = os.path.dirname(project_key['config_path'])
del project_key['config_path']

##### To allow for use of already labeled frames

In [ ]:
dlc_path = Path('/nimbus/deeplabcut/projects/scratch_test-JG_DG-2022-10-27//')
video_files = ['20201103_peanut_04_r2',
               '20210529_J16_02_r1',]

In [ ]:
dlc_path = Path('/cumulus/deeplabcut/LH_4LED_model-Daniel_Sharon-2022-07-15/')

In [ ]:
for video_file in video_files:
    h5_file = glob.glob(f'{dlc_path.as_posix()}/'
                    f'labeled-data/{video_file}/*.h5')[0]
    dlc_df = pd.read_hdf(h5_file)
    dlc_df.columns.set_levels([team_name],level=0,inplace=True)
    dlc_df.to_hdf(
        Path(
            f"{dlc_path.as_posix()}/labeled-data/{video_file}/CollectedData_{team_name}.h5").as_posix(),
        'df_with_missing')

In [ ]:
sgp.DLCProject.add_training_files({'project_name': 'scratch_test'})

In [ ]:
sgp.DLCProject.File()

#### [DLCModelTraining](#ToC)<a id='DLCModelTraining'></a>

In [ ]:
sgp.DLCModelTrainingParams.insert_new_params(paramset_name='JG_DG_training',
                                             params={'trainingsetindex': 0,
                                                     'shuffle': 1,
                                                     'gputouse': 4,
                                                     "net_type": 'resnet_50',
                                                     'augmenter_type': 'imgaug',
                                                    })

In [ ]:
sgp.DLCModelTrainingSelection()

In [ ]:
sgp.DLCModelTrainingSelection().insert1({'project_name': 'scratch_test',
                                         'dlc_training_params_name': 'JG_DG_training',
                                         'training_id': 0,
                                         'model_prefix': '',})

In [ ]:
sgp.DLCModelTraining.populate({'project_name': 'scratch_test',
                              'dlc_training_params_name': 'JG_DG_training'})

In [ ]:
model_training_key = (sgp.DLCModelTrainingSelection & {'project_name': 'scratch_test',
                                                          'dlc_training_params_name': 'JG_DG_training',}).fetch1('KEY')

#### [DLCModel](#ToC) <a id='DLCModel'></a>

In [ ]:
# sgp.DLCModelSource.insert_entry(key=model_training_key,
#                                    dlc_model_name='LH_4LED_model_2',
#                                    project_name='scratch_test',
#                                    source='FromUpstream',
#                                    skip_duplicates=True)

In [ ]:
sgp.DLCModelSource.FromUpstream()

In [ ]:
sgp.DLCModelParams().insert_default(skip_duplicates=True)

In [ ]:
sgp.DLCModelSelection().insert1({'dlc_model_name': 'LH_4LED_model_2',
                                   'dlc_model_params_name': 'default'},
                                  skip_duplicates=True)

In [ ]:
key = (sgp.DLCModelSelection & {'dlc_model_name': 'LH_4LED_model_2'}).fetch1('KEY')

In [ ]:
sgp.DLCModel.populate(key)

In [ ]:
sgp.DLCModel()

#### [DLCPoseEstimation](#ToC) <a id='DLCPoseEstimation'></a>

In [ ]:
sgp.DLCPoseEstimationSelection()

In [ ]:
sgp.DLCPoseEstimationSelection.insert_estimation_task({
    'nwb_file_name': 'J1620210604_.nwb',
    'epoch': 14,
    'video_file_num': 0,
    'project_name': 'scratch_test',
    'dlc_model_name': 'scratch_test_JG_DG_training_00',
    'dlc_model_params_name': 'default',
}, task_mode='trigger', params={'gputouse': 4, 'videotype': 'mp4'})

In [ ]:
sgp.DLCPoseEstimation().populate({'nwb_file_name': 'J1620210604_.nwb',
                                     'dlc_model_name': 'scratch_test_JG_DG_training_00',
                                     'task_mode': 'trigger'})

In [ ]:
(sgp.DLCPoseEstimation.BodyPart() & {'dlc_model_name': 'scratch_test_JG_DG_training_00',
                              'bodypart': 'greenLED',}).fetch1_dataframe()

In [ ]:
sgp.DLCPoseEstimation().BodyPart & key

In [ ]:
sgp.DLCSmoothInterpSelection().insert1(
{
    "nwb_file_name": "J1620210604_.nwb",
    "epoch": 4,
    "video_file_num": 0,
    "project_name": "scratch_test",
    "dlc_model_name": "scratch_test_JG_DG_training_00",
    "bodypart"
})

#### [DLCSmoothInterp](#ToC) <a id='DLCSmoothInterp'></a>

In [ ]:
sgp.DLCSmoothInterpParams.insert_default()

In [ ]:
sgp.DLCSmoothInterpParams().insert1({'dlc_si_params_name': 'JG_SI_params',"params":
            {"smoothing_params": {
                "smoothing_duration": 0.05,
            },
            "interp_params": {
                "likelihood_thresh": 0.95,
            },
            "max_plausible_speed": 300.0,
            "speed_smoothing_std_dev": 0.100,
            "sampling_rate": 50,
            }}, skip_duplicates=True)

In [ ]:
sgp.DLCSmoothInterpSelection.insert([{'nwb_file_name': 'J1620210604_.nwb',
                                        'epoch': 14,
                                        'video_file_num': 0,
                                         "project_name": "scratch_test",
                                        'dlc_model_name': 'scratch_test_JG_DG_training_00',
                                        'dlc_model_params_name': 'default',
                                        'bodypart': 'greenLED',
                                        'dlc_si_params_name': 'JG_SI_params',
                                        },
                                       {'nwb_file_name': 'J1620210604_.nwb',
                                        'epoch': 14,
                                        'video_file_num': 0,
                                        "project_name": "scratch_test",
                                        'dlc_model_name': 'scratch_test_JG_DG_training_00',
                                        'dlc_model_params_name': 'default',
                                        'bodypart': 'redLED_L',
                                        'dlc_si_params_name': 'JG_SI_params',
                                        },
                                       {'nwb_file_name': 'J1620210604_.nwb',
                                        'epoch': 14,
                                        'video_file_num': 0,
                                        "project_name": "scratch_test",
                                        'dlc_model_name': 'scratch_test_JG_DG_training_00',
                                        'dlc_model_params_name': 'default',
                                        'bodypart': 'redLED_C',
                                        'dlc_si_params_name': 'JG_SI_params',
                                        },
                                       {'nwb_file_name': 'J1620210604_.nwb',
                                        'epoch': 14,
                                        'video_file_num': 0,
                                        "project_name": "scratch_test",
                                        'dlc_model_name': 'scratch_test_JG_DG_training_00',
                                        'dlc_model_params_name': 'default',
                                        'bodypart': 'redLED_R',
                                        'dlc_si_params_name': 'JG_SI_params',
                                        }], skip_duplicates=True)

In [ ]:
sgp.DLCSmoothInterpSelection()

In [ ]:
sgp.DLCSmoothInterp().populate({'dlc_si_params_name': 'default',
                                  'dlc_model_name': 'scratch_test_JG_DG_training_00',})

In [ ]:
sgp.DLCSmoothInterp().populate({'dlc_si_params_name': 'JG_SI_params',
                                  'dlc_model_name': 'scratch_test_JG_DG_training_00',})

In [ ]:
sgp.DLCSmoothInterp() & {'nwb_file_name': 'J1620210604_.nwb',
                           'epoch': 14,
                           'dlc_model_name': 'scratch_test_JG_DG_training_00'}

In [ ]:
(sgp.DLCSmoothInterp() & {'nwb_file_name': 'J1620210604_.nwb',
                             'epoch': 14,
                             'dlc_model_name': 'scratch_test_JG_DG_training_00',
                             'bodypart': 'greenLED'}).fetch1_dataframe().plot.scatter(x='x',y='y', s=1)

#### [DLCSmoothInterpCohort](#ToC) <a id='DLCSmoothInterpCohort'></a>

In [ ]:
si_key = (sgp.DLCSmoothInterp() & {
    'nwb_file_name': 'J1620210604_.nwb',
    'epoch': 14,
    'dlc_model_name': 'scratch_test_JG_DG_training_00',
    'dlc_si_params_name': 'JG_SI_params',
    'bodypart': 'greenLED',}).fetch1('KEY')

In [ ]:
cohort_key = si_key.copy()
if 'bodypart' in cohort_key:
    del cohort_key['bodypart']
if 'dlc_si_params_name' in cohort_key:
    del cohort_key['dlc_si_params_name']
cohort_key['dlc_si_cohort_selection_name'] = '4LEDs'
cohort_key['bodyparts_params_dict'] = {'greenLED': 'JG_SI_params',
                                   'redLED_L':'JG_SI_params',
                                   'redLED_C':'JG_SI_params',
                                   'redLED_R':'JG_SI_params',}

In [ ]:
cohort_key

In [ ]:
sgp.DLCSmoothInterpCohortSelection().insert1(cohort_key, skip_duplicates=True)

In [ ]:
sgp.DLCSmoothInterpCohortSelection()

In [ ]:
sgp.DLCSmoothInterpCohort.populate()

In [ ]:
sgp.DLCSmoothInterpCohort.BodyPart()

In [ ]:
(sgp.DLCSmoothInterpCohort.BodyPart() & cohort_key & {'bodypart': 'greenLED'}).fetch1_dataframe()

#### [DLCCentroid](#ToC) <a id='DLCCentroid'></a>

In [ ]:
centroid_params = {
    'centroid_method': 'four_led_centroid',
    'points' : {
        'green_LED': 'greenLED',
        'red_LED_L': 'redLED_L',
        'red_LED_C': 'redLED_C',
        'red_LED_R': 'redLED_R',},
    "speed_smoothing_std_dev": 0.100,
    "sampling_rate": 50,
}
sgp.DLCCentroidParams.insert1({'dlc_centroid_params_name': 'JG_4LED',
                                'params': centroid_params},
                                skip_duplicates=True)

In [ ]:
sgp.DLCCentroidSelection()

In [ ]:
centroid_key = (sgp.DLCSmoothInterpCohort & cohort_key).fetch1('KEY')
centroid_key['dlc_centroid_params_name'] = 'JG_4LED'

In [ ]:
centroid_key

In [ ]:
sgp.DLCCentroidSelection.insert1(centroid_key, skip_duplicates=True)

In [ ]:
sgp.DLCCentroidSelection()

In [ ]:
sgp.DLCCentroid.populate()

In [ ]:
fig = (sgp.DLCCentroid() & centroid_key).fetch1_dataframe().plot.scatter(x='position_x',
                                                     y='position_y',
                                                     c='speed',
                                                     colormap='viridis',
                                                     alpha=0.5,
                                                    s=0.5,
                                                    figsize=(15,15))

#### [DLCOrientation](#ToC) <a id='DLCOrientation'></a>

In [ ]:
sgp.DLCOrientationParams().delete()

In [ ]:
sgp.DLCOrientationParams().insert_default()

In [ ]:
temp_cohort_key = (sgp.DLCSmoothInterpCohortSelection() & si_key).fetch1('KEY')

In [ ]:
temp_cohort_key

In [ ]:
fields = list(sgp.DLCOrientationSelection.fetch().dtype.fields.keys())
orient_key = {key: val for key,val in temp_cohort_key.items() if key in fields}
orient_key['dlc_orientation_params_name'] = 'default'

In [ ]:
orient_key

In [ ]:
sgp.DLCOrientationSelection().insert1(orient_key, skip_duplicates=True)

In [ ]:
sgp.DLCOrientationSelection()

In [ ]:
sgp.DLCOrientation().populate(orient_key)

In [ ]:
(sgp.DLCOrientation() & orient_key).fetch1_dataframe()

#### [DLCPos](#ToC) <a id='DLCPos'></a>

In [ ]:
sgp.DLCPos()

In [ ]:
sgp.DLCCentroid()

In [ ]:
dlc_pos_key = (sgp.DLCCentroid() & centroid_key).fetch1('KEY')

In [ ]:
dlc_pos_key.update(
    {
        'dlc_si_cohort_centroid': centroid_key['dlc_si_cohort_selection_name'],
        'dlc_si_cohort_orientation': orient_key['dlc_si_cohort_selection_name'],
        'dlc_orientation_params_name': orient_key['dlc_orientation_params_name'],
    }
)
if 'dlc_si_cohort_selection_name' in dlc_pos_key:
    del dlc_pos_key['dlc_si_cohort_selection_name']
sgp.DLCPosSelection().insert1(dlc_pos_key, skip_duplicates=True)

In [ ]:
sgp.DLCPos().populate()

In [ ]:
dlc_pos_key

In [ ]:
sgp.DLCPosVideo().populate()

#### [PosSource](#ToC) <a id='PosSource'></a>

In [ ]:
sgp.PosSource()

In [ ]:
sgp.PosSource.DLCPos()

#### [IntervalPositionInfo](#ToC) <a id='IntervalPositionInfo'></a>

In [ ]:
int_pos_info_key = (sgp.PosSource & dlc_pos_key).fetch1('KEY')

In [ ]:
sgp.IntervalPositionInfoSelection().insert1(int_pos_info_key)

In [ ]:
sgp.IntervalPositionInfo.populate()

In [ ]:
sgp.IntervalPositionInfo().fetch1_dataframe()

### [Return To Table of Contents](#ToC)<br>